* https://web3py.readthedocs.io/en/stable/quickstart.html#local-providers
* https://web3py.readthedocs.io/en/stable/examples.html#getting-the-latest-block
* https://stackoverflow.com/questions/77149484/how-can-i-generate-the-abi-of-my-smart-contract-locally-with-foundry-forge
* https://book.getfoundry.sh/reference/forge/forge-inspect

### Build
$ forge build

### Local Deployment
##### 1. Start local node
$ anvil

##### 2. Deploy testnet
$ forge script script/testnet/TestnetStage1Deploy.s.sol --rpc-url http://127.0.0.1:8545 --broadcast

##### 3. Mint
$ forge script script/testnet/TestnetStage1Deposit.s.sol --rpc-url http://127.0.0.1:8545 --broadcast 

##### 4. Capture ABI
$ forge inspect --pretty script/AddLiquidity.s.sol:AddLiquidityScript abi 

$ forge inspect script/AddLiquidity.s.sol:AddLiquidityScript abi > configs/AddLiquidityScript.json

$ forge inspect script/MintableRCStudy.s.sol:MintableRCStudyScript abi > configs/MintableRCStudyScript.json

In [ ]:
def concat_str(str0, max_len = None):
    max_len = 40 if max_len == None else max_len
    str0 = "".join([str0, " "])
    str_len = len(str0)
    return str0.ljust(max_len, ' ')

def str_output(in0, in1, max_len = None):    
    print_str_list = [concat_str(in0, max_len), str(in1)]    
    return ''.join(print_str_list)

def call_fn(contract, str_fn):
    attr = getattr(contract.functions, str_fn)
    return attr().call()

In [ ]:
from web3 import Web3
import json
from pachira import *

In [ ]:
addr_non_owner = '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266'
addr_sponsor = '0x70997970C51812dc3A010C7d01b50e0d17dc79C8'
addr_deft = '0x948B3c65b89DF0B4894ABE91E6D02FE579834F8F'

#### Connect locally

In [ ]:
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.is_connected()

#### Test: get arbitrary block

In [ ]:
block = w3.eth.get_block(6)
block

#### Test: pull tx data from block

In [ ]:
tx_hash = block.transactions[0]
tx = w3.eth.get_transaction(tx_hash)
tx

In [ ]:
w3.eth.get_balance(tx['from'])

#### Test: pull tx receipt 

In [ ]:
receipt = w3.eth.get_transaction_receipt(tx['hash'])
receipt

#### Test: decode log data

In [ ]:
log_data = receipt['logsBloom']
args = Conversion().decode_data(log_data.hex())
topics = receipt['logs']

## Study event data

In [ ]:
tx_path1 = '/Users/ian_moore/repos/indexeddefi/broadcast/AddLiquidity.s.sol/31337/run-latest.json'
tx_path2 = '/Users/ian_moore/repos/indexeddefi/broadcast/MintableRCStudy.s.sol/31337/run-latest.json'
#f = open(tx_path2)
#tx_data = json.load(f)

#### Get study ABI data

In [ ]:
study_abi_path = '/Users/ian_moore/repos/indexeddefi/out/MintableRCStudy.sol/MintableRCStudy.json'
abi_path1 = '/Users/ian_moore/repos/indexeddefi/configs/AddLiquidityScript.json'
abi_path2 = '/Users/ian_moore/repos/indexeddefi/configs/MintableRCStudyScript.json'
f = open(study_abi_path)
study_abi_data = json.load(f)

#### Pull all view functions

In [ ]:
view_fns = []; c = 0
for k, record in enumerate(study_abi_data['abi']):
    if(record['type'] == 'function' and len(record['inputs']) == 0 and record['stateMutability'] == 'view'):
        c+=1
        print(f"[{c}] View function: {record['name']}(), inputs: {record['inputs']}")
        view_fns.append(record['name'])

#### Connect to study

In [ ]:
abi_deft = json.dumps(study_abi_data['abi'])
deft_contract = w3.eth.contract(address=addr_deft, abi=abi_deft)

#### Test: view function calls

In [ ]:
print('--------------------------------------------')
print('--- View functions: MintableRCStudy.sol  ---')
print('--------------------------------------------')
for k, fn_str in enumerate(view_fns) :
    try:
        fn_res = call_fn(deft_contract, fn_str)
        fn_str = str_output(f'[{k}]', fn_str+'()', max_len = 5)
        print(str_output(fn_str, fn_res))
    except ValueError:
        print('function call error')